In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as display
import PIL.Image

In [ ]:
tf.random.set_seed(42)

In [ ]:
content_path = tf.keras.utils.get_file('lincoln','https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/gettyimages-96264452-1513966019.jpg?crop=1xw:1xh;center,top&resize=980:*')

In [ ]:
style_path = tf.keras.utils.get_file('Law of Jungle.jpg','https://www.vangoghgallery.com/img/starry_night_full.jpg')

In [ ]:
# style_path = tf.keras.utils.get_file('Piet_Mondriaan,_1930_-_Mondrian_Composition_II_in_Red,_Blue,_and_Yellow.jpg','https://upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Piet_Mondriaan%2C_1930_-_Mondrian_Composition_II_in_Red%2C_Blue%2C_and_Yellow.jpg/800px-Piet_Mondriaan%2C_1930_-_Mondrian_Composition_II_in_Red%2C_Blue%2C_and_Yellow.jpg')

In [ ]:
# content_path = tf.keras.utils.get_file('Three_Beauties_of_the_Present_Day.jpg','https://upload.wikimedia.org/wikipedia/commons/thumb/6/6b/Kitagawa_Utamaro_-_Toji_san_bijin_%28Three_Beauties_of_the_Present_Day%29From_Bijin-ga_%28Pictures_of_Beautiful_Women%29%2C_published_by_Tsutaya_Juzaburo_-_Google_Art_Project.jpg/800px-thumbnail.jpg')

In [ ]:
# content_image = tf.io.read_file(content_path)
# content_image = tf.io.decode_image(content_image,3)
# content_image = tf.image.convert_image_dtype(content_image,tf.float32)

In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image,3)
    image = tf.image.convert_image_dtype(image,tf.float32)
    return image

In [ ]:
def process_image(image):
    processed_image = tf.keras.applications.vgg19.preprocess_input(image*255)
    processed_image = tf.image.resize(processed_image, size = (600,900))
    # processed_image = tf.image.resize(processed_image, size = (224,224))
    processed_image = tf.expand_dims(processed_image,0)
    return processed_image

In [ ]:
# con_img = tf.keras.applications.vgg19.preprocess_input(content_image*255)
# con_img = tf.image.resize(con_img, size = (224,224))
# con_img = tf.expand_dims(con_img,0)

In [ ]:
vgg = tf.keras.applications.VGG19(include_top=False,pooling='avg', input_shape=(600,900,3),weights='imagenet')
vgg.trainable = False

In [ ]:
content_image = load_image(content_path)
style_image = load_image(style_path)
processed_content_image = process_image(content_image)
processed_style_image = process_image(style_image)

In [ ]:
# generated_image = tf.random.uniform([1,1024,768,3])
generated_image = tf.image.resize(content_image, size = (600,900))
generated_image = tf.expand_dims(generated_image,0)
# generated_image = tf.keras.applications.vgg16.preprocess_input(generated_image*255)

In [ ]:
content_layers = ['block4_conv2']
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
weights = [1/5,1/5,1/5,1/5,1/5]

In [ ]:
# model = keras.Model([vgg.input],vgg.get_layer('block1_conv2').output)
model = keras.Model([vgg.input],[vgg.get_layer(layer).output for layer in (style_layers + content_layers)])

In [ ]:
model.outputs

In [ ]:
def content_loss(generated_output, content_outputs):
    loss = 0.5*tf.reduce_sum(tf.square(tf.subtract(content_outputs,generated_output)))
    return loss

In [ ]:
def gram_matrix(input):
    output = tf.reshape(input,(-1,input.shape[-1]))
    output = tf.matmul(output,output,transpose_a=True)
    # Alternate implementation
    # output = tf.einsum('bijc,bijd->bcd',input,input)
    return output

In [ ]:
def style_loss(generated_outputs,style_outputs,weights):
    for generated_output,style_output,weight in zip(generated_outputs,style_outputs,weights):
        nh,nw,nc = generated_output.shape[1:]
        gram_generated_im = gram_matrix(generated_output)
        gram_style_image = gram_matrix(style_output) 
        loss = weight*(1/(4*(nh*nw*nc)))*tf.reduce_sum(tf.square(tf.subtract(gram_generated_im,gram_style_image)))
        # loss += weight*tf.reduce_sum(tf.square(tf.subtract(gram_style_image,gram_generated_im)))
        # print('Style Loss: {}'.format(loss))
    return loss

In [ ]:
def total_loss(content_loss, style_loss, alpha, beta):
    total = tf.add(tf.multiply(alpha,content_loss), tf.multiply(beta,style_loss))
    return total

In [ ]:
def train(gen_image,style_outputs,content_outputs,weights,alpha,beta):
    with tf.GradientTape() as tape:
        output_gen = model(tf.keras.applications.vgg19.preprocess_input(gen_image*255))
        # loss = 0.5*tf.reduce_sum(tf.square(tf.subtract(output_content,output_gen)))
        cont_loss = content_loss(output_gen[-1],content_outputs)
        sty_loss = style_loss(output_gen[:-1],style_outputs,weights)
        loss_total = total_loss(cont_loss,sty_loss,alpha,beta)
    # print(loss_total)
    return tape.gradient(loss_total,X)

In [ ]:
content_outputs = model(processed_content_image)[-1]
style_outputs = model(processed_style_image)[:-1]

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

In [ ]:
X = tf.Variable(generated_image)

In [ ]:
epoch = 10
steps = 100

for n in range(epoch):
    print('Epoch:{}/10\n'.format(n))
    display.display(PIL.Image.fromarray(np.array(X[0]*255, dtype = np.uint8)))
    for m in range(steps):
        # print('Step:{}/100'.format(m))
        grad = train(X,style_outputs=style_outputs,content_outputs=content_outputs,weights=weights, alpha= 1e-1, beta = 1)
        optimizer.apply_gradients(grads_and_vars=[(grad,X)])
        X.assign(tf.clip_by_value(X, clip_value_min=0.0, clip_value_max=1.0),read_value=False)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(content_image)

In [ ]:
plt.figure(figsize = (20,20))
plt.imshow(style_image)

In [ ]:
plt.figure(figsize = (20,20))
plt.imshow(X[0])